In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def generate_complex_series(n_points=1000):
    np.random.seed(42)
    time = np.arange(n_points)
    dates = pd.date_range(start='2020-01-01', periods=n_points, freq='D')
    
    # 1. TENDENCIA: Polinómica (No lineal)
    trend = 0.001 * time**1.5 
    
    # 2. ESTACIONALIDAD DUAL: Semanal (7) y Mensual (30.5)
    seasonal_weekly = 5 * np.sin(2 * np.pi * time / 7)
    seasonal_monthly = 10 * np.sin(2 * np.pi * time / 30.5)
    
    # 3. PROCESO AR(1): Memoria del dato anterior (phi = 0.7)
    ar_component = np.zeros(n_points)
    for t in range(1, n_points):
        ar_component[t] = 0.7 * ar_component[t-1] + np.random.normal(0, 1)
        
    # 4. HETEROCEDASTICIDAD: El ruido crece con el tiempo
    noise_std = 1 + 0.01 * time
    noise = np.random.normal(0, noise_std, n_points)
    
    # 5. QUIEBRE ESTRUCTURAL: Un salto súbito en el punto 700 (ej. un cambio de política)
    structural_break = np.where(time > 700, 20, 0)
    
    # ENSAMBLAJE FINAL
    y = trend + seasonal_weekly + seasonal_monthly + ar_component + noise + structural_break
    
    df = pd.DataFrame({'date': dates, 'value': y})
    df.set_index('date', inplace=True)
    return df

# Generar y visualizar
df = generate_complex_series()

plt.figure(figsize=(14, 6))
plt.plot(df.index, df.value, label='MegaPro Series (Raw Data)', color='#2E86C1')
plt.title('Dataset Maestro de Series de Tiempo para Aprendizaje Avanzado')
plt.axvline(df.index[700], color='red', linestyle='--', label='Quiebre Estructural')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(df.head())